**Sürüm 2'de yapılan değişiklikler**

1. Veri kümesi artırılmış yinelenen görüntüler içeriyor. Bu sürümde, doğrulama setinin yalnızca benzersiz görüntülerden oluştuğundan emin olmak için adımlar attım, yani tüm artırılmış görüntüler eğitim setindeydi.
2. Yerel Keras'tan tensorflow.keras'a değiştirildi.
3. Modeli bu çekirdekte Keras'tan Tensorflowjs'ye dönüştürmedim. Bunun nedeni, bir tf.keras modeli dönüştürüldüğünde uygulamada çalışmadığını keşfettim. Yerel bir Keras modeli olmalı. Ayrıca, model dönüştürme kodu önceki çekirdekte çalışıyordu ancak şimdi bir hata veriyor. Yakın zamanda yapılan bir yazılım güncellemesinin buna neden olabileceğini düşünüyorum.

***


**Giriş**

Her zaman uçtan uca bir makine öğrenimi çözümü oluşturmak istemişimdir - model oluşturmayla başlayıp canlı bir web uygulamasıyla biten. Bu projeyle birlikte, kullanıcılar bir cilt lezyonunun resmini gönderebilir ve anında bir tahmin alabilir. Bu döküman, modeli oluşturmak ve ardından onu Keras'tan Tensorflow.js'ye dönüştürmek için izlediğim süreci ayrıntılarıyla anlatıyor. Uygulamanın javascript, html ve css kodu github'da mevcuttur. <br>

Web Uygulaması:<br>http://myrista.tplinkdns.com/<br>
Github: <br>https://github.com/gms10ur/AI-Lezyon-Web<br>

Bu model cilt lezyonlarını yedi sınıfa ayırır. Özenle ayarlanmış MobileNet CNN türünde bir modeldir. Ana zorluklar, dengesiz veri seti ve az miktarda veri olmasıydı. Sınıf dengesizliğini azaltmak için veri büyütmeyi kullandım ve bunu yaparken tek bir çoğunluk sınıfı tarafından aşırı derecede çarpık olmayan kategorik doğruluk puanları elde ettim.

MobileNet'in küçük boyutu ve hızı, onu web dağıtımı için ideal hale getirir. Ayrıca, eğitmek de eğlencelidir.

Tensorflow.js, makine öğrenimi modellerinin herhangi bir ek yazılım indirmeye veya yüklemeye gerek kalmadan tarayıcıda çalışmasına izin veren yeni bir kitaplıktır. Model yerel olarak çalıştığından, kullanıcının gönderdiği hiçbir veri, bilgisayarından veya cep telefonundan asla ayrılmaz. Konu tıbbi veriler olduğunda mahremiyetin özellikle önemli olduğunu düşünüyorum.

<hr>

**Hedef nedir?**

En başta net bir hedef belirlemeyi çok yararlı buldum. Bu, model seçim sürecini yönlendirmeye yardımcı olur. Örneğin, bir modelin doğruluğu %60 ise, genellikle kötü bir model olarak görülür. Bununla birlikte, aynı zamanda %90'lık bir ilk 3 doğruluğa sahipse ve amaç, 3 tahmin vermesini gerektiriyorsa, aslında oldukça iyi bir model olabilir.

*Bu görev için tanımladığım hedef bu:*

> Doktorlara ve laboratuvar teknologlarına belirli bir cilt lezyonu için en yüksek olasılıklı üç tanıyı söyleyebilecek çevrimiçi bir araç oluşturun. Bu, yüksek öncelikli hastaları hızla belirlemelerine ve iş akışlarını hızlandırmalarına yardımcı olacaktır. Uygulamanın 3 saniyeden daha kısa sürede sonuç vermesi gerekir. Gizliliği sağlamak için görüntüler yerel olarak önceden işlenmeli ve analiz edilmeli ve asla harici bir sunucuya yüklenmemelidir.


In [ ]:
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np
#import keras
#from keras import backend as K

import tensorflow
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline


**LEZYON TÜRLERİ**<br>

Makaleden alıntılar:<br>
> THAM10000 Veri Kümesi: Yaygın Pigmentli Deri Lezyonlarının Çok Kaynaklı Dermatoskopik Görüntülerinin Geniş Bir Koleksiyonu<br>
https://arxiv.org/abs/1803.10417



 **nv**<br>
Melanositik nevüsler, melanositlerin iyi huylu neoplazmalarıdır ve hepsi serimize dahil edilen sayısız varyantta görülür. Varyantlar, dermatoskopik bir bakış açısından önemli ölçüde farklılık gösterebilir.<br>
 *[6705 görsel]*
 
 **mel**<br>
 Melanom, farklı varyantlarda görülebilen melanositlerden türetilen malign bir neoplazmdır. Erken bir aşamada eksize edilirse, basit cerrahi eksizyonla tedavi edilebilir. Melanomlar invaziv veya non-invaziv (yerinde) olabilir. Tüm melanom varyantlarını dahil ettik, ancak pigmentsiz, subungual, oküler veya mukozal melanomu hariç tuttuk.<br>*[1113 görsel]*
 
 
**bkl**<br>
 "Benign keratoz", seboreik keratoz ("yaşlılık siğil"), solar lentigo - seboreik keratozun düz bir varyantı olarak kabul edilebilir - ve liken-planus benzeri keratozları (LPLK) içeren genel bir sınıftır. keratoz veya iltihaplı güneş lentigo
ve gerileme [22]. Üç alt grup dermatoskopik olarak farklı görünebilir, ancak biyolojik olarak benzer oldukları ve histopatolojik olarak sıklıkla aynı jenerik terim altında rapor edildikleri için bunları birlikte grupladık. Dermatoskopik bir bakış açısından, liken planus benzeri keratozlar, melanomu taklit eden morfolojik özellikler gösterebildikleri [23] ve tanısal nedenlerle sıklıkla biyopsi veya eksize edildikleri için özellikle zorlayıcıdır.<br>
*[1099 görsel]*

**bcc**<br>
Bazal hücreli karsinom, nadiren metastaz yapan ancak tedavi edilmezse yıkıcı bir şekilde büyüyen epitelyal cilt kanserinin yaygın bir çeşididir. Hepsi bu kümeye dahil olan farklı morfolojik varyantlarda (düz, nodüler, pigmentli, kistik, vb.) [21] görülür.<br>
*[514 görsel]*
 
**akiec**<br>
Aktinik Keratoz (Solar Keratoz) ve intraepitelyal Karsinom (Bowen hastalığı) yaygın non-invaziv, skuamöz hücreli karsinomun cerrahi olmadan lokal olarak tedavi edilebilen varyantlarıdır. Bazı yazarlar onları gerçek karsinomlar olarak değil, skuamöz hücreli karsinomların öncüleri olarak görmektedir. Bununla birlikte, bu lezyonların, genellikle pigmente olmayan invaziv skuamöz hücreli karsinomaya ilerleyebileceği konusunda fikir birliği vardır. Her iki neoplazma da yaygın olarak yüzey ölçeklemesi gösterir ve genellikle pigment içermez. Aktinik keratoz yüzde daha yaygındır ve Bowen hastalığı vücudun diğer bölgelerinde daha yaygındır. Her iki tip de UV ışığı tarafından tetiklendiğinden, UV tarafından değil de insan papilloma virüsü enfeksiyonunun neden olduğu Bowen hastalığı durumları dışında, çevreleyen cilt genellikle ciddi güneş hasarı ile karakterize edilir. Bowen hastalığı [19] ve aktinik keratoz [20] için pigmentli varyantlar mevcuttur. Her ikisi de bu sete dahildir.<br>*[327 görsel]*


**vasc**<br>
Veri setindeki vasküler cilt lezyonları, kiraz anjiyomlarından anjiyokeratomlara [25] ve piyojenik granülomlara [26] kadar uzanır. Kanama da bu kategoriye dahildir.<br>
*[142 görsel]*

**df**<br>
Dermatofibroma, iyi huylu bir proliferasyon veya minimal travmaya karşı inflamatuar bir reaksiyon olarak kabul edilen iyi huylu bir cilt lezyonudur. Dermatoskopik olarak sıklıkla merkezi bir fibrozis zonu gösteren kahverengidir [24].<br>*[115 görsel]*


<br>*[Toplam görsel = 10015]*

In [ ]:
os.listdir('../input')

### Dizin yapısını oluşturma

Bu klasörlerde, daha sonra Keras oluşturucularına beslenecek olan görüntüleri saklayacağız.

In [ ]:

# Yeni bir dizin oluştur
base_dir = 'base_dir'
os.mkdir(base_dir)


#[TEMEL DİZİN İÇİNDE KLASÖRLER OLUŞTUR]

# şimdi 'base_dir' içinde 7 klasör oluşturuyoruz:

# train_dir
    # nv
    # mel
    # bkl
    # bcc
    # akiec
    # vasc
    # df
 
# val_dir
    # nv
    # mel
    # bkl
    # bcc
    # akiec
    # vasc
    # df

# yeni klasörlerin adlarını birleştireceğimiz bir 'base_dir' yolu oluştur
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# val_dir
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)


# Her klasörün içinde her sınıf için ayrı klasörler oluşturuyoruz

# train_dir içinde yeni klasörler oluştur
nv = os.path.join(train_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(train_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(train_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(train_dir, 'bcc')
os.mkdir(bcc)
akiec = os.path.join(train_dir, 'akiec')
os.mkdir(akiec)
vasc = os.path.join(train_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(train_dir, 'df')
os.mkdir(df)



# val_dir içinde yeni klasörler oluştur
nv = os.path.join(val_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(val_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(val_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(val_dir, 'bcc')
os.mkdir(bcc)
akiec = os.path.join(val_dir, 'akiec')
os.mkdir(akiec)
vasc = os.path.join(val_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(val_dir, 'df')
os.mkdir(df)


### Eğitim ve Değer Setleri Oluşturma

In [ ]:
df_data = pd.read_csv('../input/HAM10000_metadata.csv')

df_data.head()

### Katmanlı bir değer seti oluşturma

In [ ]:
# bu bize her bir lezyon_id ile kaç tane görüntünün ilişkilendirildiğini söyleyecektir.
df = df_data.groupby('lesion_id').count()

# şimdi, kendisiyle ilişkilendirilmiş yalnızca bir görüntüye sahip olan lezyon_id'leri filtreliyoruz
df = df[df['image_id'] == 1]

df.reset_index(inplace=True)

df.head()

In [ ]:
# burada, yinelenen görüntülere sahip olan ve yalnızca bir görüntüye sahip olan lezyon_id'leri tanımlarız.

def identify_duplicates(x):
    
    unique_list = list(df['lesion_id'])
    
    if x in unique_list:
        return 'no_duplicates'
    else:
        return 'has_duplicates'
    
# lesson_id sütununun bir kopyası olan yeni bir sütun oluştur
df_data['duplicates'] = df_data['lesion_id']
# işlevi bu yeni sütuna uygula
df_data['duplicates'] = df_data['duplicates'].apply(identify_duplicates)

df_data.head()

In [ ]:
df_data['duplicates'].value_counts()

In [ ]:
# şimdi kopyaları olmayan resimleri filtreliyoruz
df = df_data[df_data['duplicates'] == 'no_duplicates']

df.shape

In [ ]:
# şimdi df kullanarak bir val seti oluşturuyoruz çünkü 
# bu görüntülerin hiçbirinin eğitim setinde çoğaltılmış kopyaları olmadığından eminiz
y = df['dx']

_, df_val = train_test_split(df, test_size=0.17, random_state=101, stratify=y)

df_val.shape

In [ ]:
df_val['dx'].value_counts()

### Değer setinde yer alan görselleri hariç tutan bir eğitim seti oluşturma

In [ ]:
# Bu küme, val kümesindeki tüm satırlar hariç tutulduğunda df_data olacaktır.

# Bu işlev, bir görüntünün eğitimin mi yoksa değer kümesinin bir parçası mı olduğunu tanımlar.
def identify_val_rows(x):
    # val setindeki tüm lesion_id'lerin bir listesini oluştur
    val_list = list(df_val['image_id'])
    
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# eğitim ve val satırlarını tanımla

# resim kimliği sütununun bir kopyası olan yeni bir sütun oluştur
df_data['train_or_val'] = df_data['image_id']
# işlevi bu yeni sütuna uygula
df_data['train_or_val'] = df_data['train_or_val'].apply(identify_val_rows)
   
# eğitim satırlarını filtrele
df_train = df_data[df_data['train_or_val'] == 'train']


print(len(df_train))
print(len(df_val))

In [ ]:
df_train['dx'].value_counts()

In [ ]:
df_val['dx'].value_counts()

### Görüntüleri Klasörlere Aktarma

In [ ]:
# image_id'yi df_data'da dizin olarak ayarla
df_data.set_index('image_id', inplace=True)

In [ ]:
# İki klasörün her birindeki görüntülerin bir listesini al
folder_1 = os.listdir('../input/ham10000_images_part_1')
folder_2 = os.listdir('../input/ham10000_images_part_2')

# eğitim ve val görsellerinin listesini al
train_list = list(df_train['image_id'])
val_list = list(df_val['image_id'])



# Eğitim görüntülerini aktar

for image in train_list:
    
    fname = image + '.jpg'
    label = df_data.loc[image,'dx']
    
    if fname in folder_1:
        # görüntüye giden kaynak yolu
        src = os.path.join('../input/ham10000_images_part_1', fname)
        # görüntüye giden hedef yol
        dst = os.path.join(train_dir, label, fname)
        # görüntüyü kaynaktan hedefe kopyala
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # görüntüye giden kaynak yolu
        src = os.path.join('../input/ham10000_images_part_2', fname)
        # görüntüye giden hedef yol
        dst = os.path.join(train_dir, label, fname)
        # görüntüyü kaynaktan hedefe kopyala
        shutil.copyfile(src, dst)


# Val görüntülerini aktar

for image in val_list:
    
    fname = image + '.jpg'
    label = df_data.loc[image,'dx']
    
    if fname in folder_1:
        # görüntüye giden kaynak yolu
        src = os.path.join('../input/ham10000_images_part_1', fname)
        # görüntüye giden hedef yol
        dst = os.path.join(val_dir, label, fname)
        # görüntüyü kaynaktan hedefe kopyala
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # görüntüye giden kaynak yolu
        src = os.path.join('../input/ham10000_images_part_2', fname)
        # görüntüye giden hedef yol
        dst = os.path.join(val_dir, label, fname)
        # görüntüyü kaynaktan hedefe kopyala
        shutil.copyfile(src, dst)
        

In [ ]:
# her klasörde kaç tane eğitim resmimiz olduğunu kontrol et

print(len(os.listdir('base_dir/train_dir/nv')))
print(len(os.listdir('base_dir/train_dir/mel')))
print(len(os.listdir('base_dir/train_dir/bkl')))
print(len(os.listdir('base_dir/train_dir/bcc')))
print(len(os.listdir('base_dir/train_dir/akiec')))
print(len(os.listdir('base_dir/train_dir/vasc')))
print(len(os.listdir('base_dir/train_dir/df')))

In [ ]:
# her klasörde kaç tane val resmimiz olduğunu kontrol et

print(len(os.listdir('base_dir/val_dir/nv')))
print(len(os.listdir('base_dir/val_dir/mel')))
print(len(os.listdir('base_dir/val_dir/bkl')))
print(len(os.listdir('base_dir/val_dir/bcc')))
print(len(os.listdir('base_dir/val_dir/akiec')))
print(len(os.listdir('base_dir/val_dir/vasc')))
print(len(os.listdir('base_dir/val_dir/df')))

### Eğitim resimlerini aug_dir içine kopyalayın

In [ ]:
# 'nv' sınıfını artırmadığımızı unutma
class_list = ['mel','bkl','bcc','akiec','vasc','df']

for item in class_list:
    
    # Bu dizinleri daha sonra sildiğimiz için burada geçici dizinler oluşturuyoruz
    # bir temel dizin oluştur
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)
    # aynı sınıfın görüntülerini depolamak için temel dizinde bir dizin oluştur
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    # Bir sınıf seç
    img_class = item

    # o dizindeki tüm resimleri listele
    img_list = os.listdir('base_dir/train_dir/' + img_class)

    # Görüntüleri sınıf train dizininden img_dir'e kopyalayın, ör. 'mel' sınıfı
    for fname in img_list:
            # görüntüye giden kaynak yolu
            src = os.path.join('base_dir/train_dir/' + img_class, fname)
            # görüntüye giden hedef yol
            dst = os.path.join(img_dir, fname)
            # görüntüyü kaynaktan hedefe kopyalayın
            shutil.copyfile(src, dst)


    # görüntülerin kendilerine değil, görüntüleri içeren bir dizine işaret et
    path = aug_dir
    save_path = 'base_dir/train_dir/' + img_class

    # Bir veri oluşturucu oluştur
    datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        #brightness_range=(0.9,1.1),
        fill_mode='nearest')

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpg',
                                                    target_size=(224,224),
                                                    batch_size=batch_size)



    # Artırılmış görüntüleri oluşturun ve bunları eğitim klasörlerine ekle
    
    ###########
    
    num_aug_images_wanted = 6000 # her sınıfta olmasını istediğimiz toplam resim sayısı
    
    ###########
    
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

    # oluşturucuyu çalıştır ve yaklaşık 6000 artırılmış görüntü oluştur
    for i in range(0,num_batches):

        imgs, labels = next(aug_datagen)
        
    # ham görüntü dosyalarıyla geçici dizini sil
    shutil.rmtree('aug_dir')

In [ ]:
# Artık her klasörde kaç tane eğitim resmimiz olduğunu kontrol et
# Bu, orijinal görüntüler ve artırılmış görüntüler

print(len(os.listdir('base_dir/train_dir/nv')))
print(len(os.listdir('base_dir/train_dir/mel')))
print(len(os.listdir('base_dir/train_dir/bkl')))
print(len(os.listdir('base_dir/train_dir/bcc')))
print(len(os.listdir('base_dir/train_dir/akiec')))
print(len(os.listdir('base_dir/train_dir/vasc')))
print(len(os.listdir('base_dir/train_dir/df')))

In [ ]:
# Her klasörde kaç tane değer resmimiz olduğunu kontrol et

print(len(os.listdir('base_dir/val_dir/nv')))
print(len(os.listdir('base_dir/val_dir/mel')))
print(len(os.listdir('base_dir/val_dir/bkl')))
print(len(os.listdir('base_dir/val_dir/bcc')))
print(len(os.listdir('base_dir/val_dir/akiec')))
print(len(os.listdir('base_dir/val_dir/vasc')))
print(len(os.listdir('base_dir/val_dir/df')))

### 50 artırılmış görüntüyü görselleştir

In [ ]:
# resimleri jüpyter not defterinde etiketlerle çizer
# kaynak: https://github.com/smileservices/keras_utils/blob/master/utils.py

def plots(ims, figsize=(12,6), rows=5, interp=False, titles=None): # 12,6
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
        
plots(imgs, titles=None) # titles=labels resim etiketlerini gösterecek

In [ ]:
# Veri Hazırlığının Sonu
### ===================================================================================== ###
# Model Oluşturmanın Başlangıcı

### Oluşturucuları Ayarla

In [ ]:
train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)


In [ ]:

datagen = ImageDataGenerator(
    preprocessing_function= \
    tensorflow.keras.applications.mobilenet.preprocess_input)

train_batches = datagen.flow_from_directory(train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size)

valid_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size)

# Not: shuffle=False, test veri setinin karıştırılmamasına neden olur
test_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)

### MobileNet Modelini Değiştir

In [ ]:
# mobilenet modelinin bir kopyasını oluştur

mobile = tensorflow.keras.applications.mobilenet.MobileNet()

In [ ]:
mobile.summary()

In [ ]:
type(mobile.layers)

In [ ]:
# MobileNet'in kaç katmanı var?
len(mobile.layers)

In [ ]:
# MODEL MİMARİSİNİ OLUŞTUR

# Yukarıdaki modelin son 5 katmanını hariç tut.
# Bu, global_average_pooling2d_1'e kadar olan tüm katmanları içerecektir
x = mobile.layers[-6].output

# Tahminler için yeni bir yoğun katman oluştur
# 7 sınıf sayısına karşılık gelir
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

# inputs=mobile.input giriş katmanını seçer 
# outputs=predictions yukarıda oluşturduğumuz yoğun katmanı ifade eder.

model = Model(inputs=mobile.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
# Aslında kaç katman eğitilmek istediğimizi seçmemiz gerekiyor.

# Burada yeni modeldeki son 23 katman hariç
# tüm katmanların ağırlıklarını donduruyoruz.
# Modelin son 23 katmanı eğitilecektir.

for layer in model.layers[:-23]:
    layer.trainable = False

### Modeli Eğitmek

In [ ]:
# Top2 ve Top3 Doğruluğunu Tanımla

from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [ ]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', 
              metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])



In [ ]:
# Her dizinle ilişkili etiketleri al
print(valid_batches.class_indices)

In [ ]:
# Modeli melanomaya daha duyarlı hale getirmek için ağırlıklar ekle

class_weights={
    0: 1.0, # akiec
    1: 1.0, # bcc
    2: 1.0, # bkl
    3: 1.0, # df
    4: 3.0, # mel # Modeli Melanom'a karşı daha duyarlı hale getirmeye çalış.
    5: 1.0, # nv
    6: 1.0, # vasc
}

In [ ]:

filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit_generator(train_batches, steps_per_epoch=train_steps, 
                              class_weight=class_weights,
                    validation_data=valid_batches,
                    validation_steps=val_steps,
                    epochs=30, verbose=1,
                   callbacks=callbacks_list)


### Val setini kullanarak modeli değerlendir

In [ ]:
# evaulate_generator'ı kullanabilmemiz için metrik adlarını alın
model.metrics_names

In [ ]:
# Burada son sonuç kullanılacaktır.

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

In [ ]:
# Burada en iyi sonuç kullanılacaktır.

model.load_weights('model.h5')

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

### Eğitim Eğrilerini Çizin

In [ ]:
# kayıp ve doğruluk eğrilerini göster

import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
train_top2_acc = history.history['top_2_accuracy']
val_top2_acc = history.history['val_top_2_accuracy']
train_top3_acc = history.history['top_3_accuracy']
val_top3_acc = history.history['val_top_3_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training cat acc')
plt.plot(epochs, val_acc, 'b', label='Validation cat acc')
plt.title('Training and validation cat accuracy')
plt.legend()
plt.figure()


plt.plot(epochs, train_top2_acc, 'bo', label='Training top2 acc')
plt.plot(epochs, val_top2_acc, 'b', label='Validation top2 acc')
plt.title('Training and validation top2 accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, train_top3_acc, 'bo', label='Training top3 acc')
plt.plot(epochs, val_top3_acc, 'b', label='Validation top3 acc')
plt.title('Training and validation top3 accuracy')
plt.legend()


plt.show()

### Bir Kafa Karışıklığı Matrisi Oluştur

In [ ]:
# Test görüntülerinin etiketlerini al.
# Kedilerin ve köpeklerin ayrı klasörlerde olduğunu unutmayın, bu nedenle aşağıdaki kod
# görüntünün bulunduğu klasöre bağlı olarak etiketleri alabilir.
test_labels = test_batches.classes

In [ ]:
# Karışıklık matrisini çizmek için bunlara ihtiyacımız var.
test_labels

In [ ]:
# Her sınıfla ilişkili etiketi yazdır
test_batches.class_indices

In [ ]:
# bir tahminde bulun
predictions = model.predict_generator(test_batches, steps=len(df_val), verbose=1)

In [ ]:
predictions.shape

In [ ]:
# Kaynak: Scikit Learn
# http://scikit-learn.org/stable/auto_examples/
# model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-
# selection-plot-confusion-matrix-py


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()




In [ ]:
test_labels.shape

In [ ]:
# argmax, bir satırdaki maksimum değerin dizinini döndürür
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
# Sınıf indekslerinin etiketlerini tanımlayın.
# Bunların yukarıda gösterilen sıraya uyması gerekir.
cm_plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel','nv', 'vasc']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [ ]:
# Model Oluşturma Sonu
### ===================================================================================== ###
# Modeli Keras'tan Tensorflow.js'ye Dönüştürme

### Tensorflow.js'yi yükleme

In [ ]:
#!pip install tensorflowjs

### Modeli Keras'tan Tensorflow.js'ye Dönüştürme
Aşağıdaki dönüştürme kodu artık kaggle çekirdeklerinde çalışmıyor. Referans olması için bıraktım.

Bu modeli dönüştürmek için geçici çözüm aşağıdaki gibidir:<br>
1. Yerel Keras'ı kullanarak modeli yeniden oluşturun.<br>
2. Modeli Keras'tan Tensorflowjs'ye dönüştürmek için komut satırı dönüştürme işlemini kullanın.<br>
Bunu nasıl yapacağınız açıklanmıştır: https://www.youtube.com/watch?v=Kc2_x6pBYGE

Yukarıdaki adımlar bir kaggle çekirdeğinde oldukça kolay bir şekilde yapılabilir. Tensorflowjs hala oldukça yeni olduğundan bu tür hatalar olağandışı değil.

In [ ]:
# model dosyalarını depolamak için bir dizin oluştur
#os.mkdir('tfjs_dir')

# Tensorflow.js'ye dönüştür
#import tensorflowjs as tfjs

# Error
# AttributeError: module 'tensorflow.python.data.ops.dataset_ops' 
    # has no attribute 'UnaryDataset'

#tfjs.converters.save_keras_model(model, 'tfjs_dir')

In [ ]:
# modeli içeren dizini kontrol et
#!ls

In [ ]:
# tensorflow.js modelini oluşturan dosyaları görüntüle
#os.listdir('tfjs_dir')

In [ ]:
# Kaggle hatasını önlemek için oluşturduğumuz görüntü veri dizinini sil.
# Kaggle, maksimum 500 dosyanın kaydedilmesine izin verir.

shutil.rmtree('base_dir')

### Kaynaklar

Bunlar kullandığım bazı kaynaklar:

1. Deeplizard tarafından Mobilenet'in Tensorflow.js ile nasıl kullanılacağına dair mükemmel eğitim serisi<br>
https://www.youtube.com/watch?v=HEQDRWMK6yY

2. Doğruluk, Kesinlik ve F1 Skoru ile ilgili Minsuk Heo'nun Öğreticisi<br>
https://www.youtube.com/watch?v=HBi-P5j0Kec

3. Data School tarafından bir sınıflandırıcının nasıl değerlendirileceğine ilişkin eğitim<br>
https://www.youtube.com/watch?v=85dtiMz9tSo

3. Tensorflow.js proje galerisi<br>
https://github.com/tensorflow/tfjs/blob/master/GALLERY.md

